In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
from ResNet import *
%matplotlib inline

In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
#root = r'/home/zz2496/Notebooks/CSRNet/shanghai'
root = r'/home/zz2496/Notebooks/CSRNet/UCF'

In [4]:
#now generate the ShanghaiA's ground truth
#part_A_train = os.path.join(root,'part_A_final/train_data','images')
#part_A_test = os.path.join(root,'part_A_final/test_data','images')
#part_B_train = os.path.join(root,'part_B_final/train_data','images')
#part_B_test = os.path.join(root,'part_B_final/test_data','images')
#path_sets = [part_A_test]

In [5]:
#train = os.path.join(root,'train_data','images')
test = os.path.join(root,'test_data','images')
path_sets = [test]

In [6]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [7]:
#train = os.path.join(root,'train_data','images')
#test = 

In [8]:
img_paths

['/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/46.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/45.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/47.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/42.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/44.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/43.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/49.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/41.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/48.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/50.jpg']

In [9]:
#model = CSRNet()
model = ResNet(BasicBlock, [3, 4, 6, 3])

In [10]:
model = model.cuda()

In [11]:
checkpoint = torch.load('0model_best_UCF_res.pth.tar')

In [12]:
model.load_state_dict(checkpoint['state_dict'])

In [13]:
#img = 255.0 * F.to_tensor(Image.open(img_paths[0]).convert('RGB'))

In [14]:
mae = 0
for i in xrange(len(img_paths)):
#    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB'))
#    img[0,:,:]=img[0,:,:]-92.8207477031
#    img[1,:,:]=img[1,:,:]-95.2757037428
#    img[2,:,:]=img[2,:,:]-104.877445883 #Q7: what are these? #change to imagenet mean
#    img = img.cuda()
    img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print i,mae
print mae/len(img_paths)

0 44.8111572265625
1 168.829833984375
2 543.1644287109375
3 595.6698303222656
4 749.7125549316406
5 1348.8564758300781
6 1615.1889343261719
7 1633.8735046386719
8 2294.9156188964844
9 2450.547607421875
245.0547607421875


In [16]:
glob.glob(os.path.join(test, '*.jpg'))

['/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/46.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/45.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/47.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/42.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/44.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/43.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/49.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/41.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/48.jpg',
 '/home/zz2496/Notebooks/CSRNet/UCF/test_data/images/50.jpg']

In [18]:
a = []
for gd in glob.glob(os.path.join(test, '*.jpg')):
    mat = io.loadmat(gd.replace('.jpg','_ann.mat').replace('images','ground_truth'))
    a.append(len(mat['annPoints']))
    print(len(mat['annPoints']))


1366
1451
1961
329
2358
947
588
1284
2372
440


In [22]:
np.mean(a)

1309.6